In [4]:
using Pkg
Pkg.activate("./")

# idea: we don't represent ssets using natural numbers but EVERYTHING is properly surreal
# otherwise, it's really hard to to the maths
# futhermore, we directly use symbolics for rewriting

include("./src2/include.jl")

const S0 = Surreal(nil, nil)
S1 = Surreal(S0, nil)

@assert S0 == S0

S2 = Surreal(S1, nil)

@assert S2 == Surreal([S0, S1], nil)

  Activating project at `~/code/Surreal.jl`


In [5]:
max(S0, S1) == S1

true

LoadError: UndefVarError: `version` not defined

In [6]:
using Symbolics
using SymbolicUtils
using SymbolicUtils: Symbolic

In [7]:
# binary ops that return Bool

for (f, Domain) in [(==) => Surreal, (!=) => Surreal,
                    (<=) => Surreal,   (>=) => Surreal,
                    (isless) => Surreal,
                    (<) => Surreal,   (> ) => Surreal,
                    (<=) => SurrealSet, (<) => SurrealSet]
    @eval begin
        promote_symtype(::$(typeof(f)), ::Type{<:$Domain}, ::Type{<:$Domain}) = Bool
        (::$(typeof(f)))(a::Symbolic{<:$Domain}, b::$Domain) = term($f, a, b, type=Bool)
        (::$(typeof(f)))(a::Symbolic{<:$Domain}, b::Symbolic{<:$Domain}) = term($f, a, b, type=Bool)
        (::$(typeof(f)))(a::$Domain, b::Symbolic{<:$Domain}) = term($f, a, b, type=Bool)
    end
end



In [8]:
@syms S(l::SurrealSet, r::SurrealSet)::Surreal 
@syms ∅::SurrealSet # empty set
@syms SSS(x::Surreal)::SurrealSet # SingularSurrealSet
@syms SN(x::Number)::Surreal # natural number
@syms maxUnion(x::SurrealSet, y::SurrealSet)::SurrealSet
@syms minUnion(x::SurrealSet, y::SurrealSet)::SurrealSet
@syms add(x::SurrealSet, y::Surreal)::SurrealSet
@syms ⊕(x::Surreal, y::Surreal)::Surreal
@syms myIfElse(c::Any, x::Surreal, y::Surreal)::Surreal

compareRules = [    
    @rule (~x <= ∅) => true
    @rule (∅ <= ~x) => true
    @rule (S(~XL, ~XR) <= S(~YL, ~YR)) => ((~XL < SSS(S(~YL, ~YR))) & (~XL < SSS(S(~YL, ~YR))))
    
    @rule (SSS(~x) <= SSS(~y)) => (~x <= ~y)
    @rule (SN(~x) <= SN(~y)) => (~x <= ~y)
]

logicRules = [
    @rule myIfElse(true, ~x, ~y) => ~x
    @rule myIfElse(false, ~x, ~y) => ~y
    
    @rule !true => false
    @rule !false => true
    @rule true & true => true
    @rule true & false => false
    @rule false & true => false
    @rule false & false => false
    
    @rule (~x < ~y) => ((~x <= ~y) & !(~y <= ~x))
    @rule (~x > ~y) => (~y < ~x)
    @rule (~x >= ~y) => (~y <= ~x)
    @rule (~x == ~y) => ((~y <= ~x) & (~x <= ~y))
    @rule (~x != ~y) => !(~x == ~y)
]

removeNames = [
    @rule SN(0) => S(∅, ∅)
    @rule SN(~x::(x -> x isa Integer && x >= 0)) => S(SSS(SN(~x - 1)), ∅)
]


unionRules = [
    @rule maxUnion(∅, ~y) => ~y
    @rule maxUnion(~x, ∅) => ~x
    @rule maxUnion(SSS(~x), SSS(~y)) => SSS(myIfElse(~x <= ~y, ~y, ~x))
    
    @rule minUnion(∅, ~y) => ~y
    @rule minUnion(~x, ∅) => ~x
    @rule minUnion(SSS(~x), SSS(~y)) => SSS(myIfElse(~x <= ~y, ~x, ~y))
    
    #@rule (S(~XL, ~XR) ⊔ S(~YL, ~YR)) => ∅
]

additionRules = [
    @rule add(∅, ~y) => ∅
    @rule add(~x, S(∅, ∅)) => ~x
    @rule add(SSS(~x), ~y) => SSS(~x ⊕ ~y)
        
    @rule (S(∅, ∅) ⊕ ~y) => ~y
    @rule (~x ⊕ S(∅, ∅)) => ~x
    @rule (S(~XL, ~XR) ⊕ S(~YL, ~YR)) =>  S(maxUnion(add(~XL, S(~YL, ~YR)), add(~YL, S(~XL, ~XR))), minUnion(add(~XR, S(~YL, ~YR)), add(~YR, S(~XL, ~XR))))
]


addNames = [
    @rule S(∅, ∅) => SN(0)
    @rule S(SSS(SN(~x::(x -> x isa Integer && x >= 0))), ∅) => SN(~x + 1)

]


cas = vcat(
    #removeNames..., 
    unionRules...,
    additionRules..., 
    #logicRules...,
    #compareRules..., 
    #addNames...
    )

prepareChain(cas) = x -> simplify(x;
        threaded=true,
        rewriter= (cas |>
        SymbolicUtils.RestartedChain |> # RestartedChain
        SymbolicUtils.Prewalk)
    )
    #=
    cas |>
    SymbolicUtils.RestartedChain |>
    (x -> SymbolicUtils.Prewalk(x; threaded=true)) |>
    SymbolicUtils.Fixpoint=#
  
    
c0 = prepareChain(removeNames)
c1 = prepareChain(cas)
c2 = prepareChain(addNames)
    
res = c1(c0(
    SN(0) ⊕ SN(1)
))
c2(res)

SN(1)

In [9]:
@time res = SN(2) ⊕ SN(2) |> c0 
@time res = res |> c1 
@show res
@time res = res |> c2
@show res

  0.002801 seconds (1.85 k allocations: 114.497 KiB, 91.74% compilation time)
  1.741133 seconds (1.84 M allocations: 117.720 MiB, 1.60% gc time, 98.93% compilation time)
res = S(SSS(myIfElse(S(SSS(myIfElse(S(SSS(S(SSS(S(∅, ∅)), ∅)), ∅) <= S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅))), ∅), S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅))), ∅), S(SSS(S(SSS(S(∅, ∅)), ∅)), ∅))), ∅) <= S(SSS(myIfElse(S(SSS(S(SSS(S(∅, ∅)), ∅)), ∅) <= S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅))), ∅), S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅))), ∅), S(SSS(S(SSS(S(∅, ∅)), ∅)), ∅))), ∅), S(SSS(myIfElse(S(SSS(S(SSS(S(∅, ∅)), ∅)), ∅) <= S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), ∅))), ∅), S(SSS(myIfElse(S(SSS(S(∅, ∅)), ∅) <= S(SSS(S(∅, ∅)), ∅), S(SSS(S(∅, ∅)), 

S(SSS(myIfElse(S(SSS(myIfElse(SN(2) <= S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), SN(2))), ∅) <= S(SSS(myIfElse(SN(2) <= S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), SN(2))), ∅), S(SSS(myIfElse(SN(2) <= S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), SN(2))), ∅), S(SSS(myIfElse(SN(2) <= S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), S(SSS(myIfElse(SN(1) <= SN(1), SN(1), SN(1))), ∅), SN(2))), ∅))), ∅)